In [2]:
import pandas as pd

# Load data
xls1 = pd.ExcelFile("E:\\数学建模国赛2023选题\\C题\\第一题数据预处理结果(针对品类销售性).xlsx")
df1=pd.read_excel(xls1)

In [3]:
df1

,Unnamed: 0,销售日期,扫码销售时间,销量(千克),销售单价(元/千克),单品名称,分类名称
0,0,2020-07-01,09:15:07.924,0.396,7.6,泡泡椒(精品),辣椒类
1,1,2020-07-01,09:17:27.295,0.849,3.2,大白菜,花叶类
2,2,2020-07-01,09:17:33.905,0.409,7.6,泡泡椒(精品),辣椒类
3,3,2020-07-01,09:19:45.450,0.421,10.0,上海青,花叶类
4,4,2020-07-01,09:20:23.686,0.539,8.0,菜心,花叶类
...,...,...,...,...,...,...,...
878498,878498,2023-06-30,21:35:13.264,0.284,24.0,西峡花菇(1),食用菌
878499,878499,2023-06-30,21:35:14.358,0.669,12.0,长线茄,茄类
878500,878500,2023-06-30,21:35:20.264,0.125,24.0,西峡花菇(1),食用菌
878501,878501,2023-06-30,21:35:21.509,0.252,5.2,芜湖青椒(1),辣椒类


In [4]:
df1.drop(columns=['Unnamed: 0','销售单价(元/千克)'], inplace=True, errors='ignore')

# 显示前几行以确认更改
df1.head()

,销售日期,扫码销售时间,销量(千克),单品名称,分类名称
0,2020-07-01,09:15:07.924,0.396,泡泡椒(精品),辣椒类
1,2020-07-01,09:17:27.295,0.849,大白菜,花叶类
2,2020-07-01,09:17:33.905,0.409,泡泡椒(精品),辣椒类
3,2020-07-01,09:19:45.450,0.421,上海青,花叶类
4,2020-07-01,09:20:23.686,0.539,菜心,花叶类


In [5]:
# 将'销售日期'和具体的时间字段（如'销售时间'）合并为一个完整的时间戳
df1['完整时间戳'] = pd.to_datetime(df1['销售日期'].astype(str) + ' ' + df1['扫码销售时间'].astype(str))

# 按照分钟和商品进行分组，并对销量进行求和
transaction_data_by_minute = df1.groupby([pd.Grouper(key='完整时间戳', freq='T'), '分类名称'])['销量(千克)'].sum().unstack().fillna(0)

# 将销量转换为0和1（0表示未购买，1表示购买）
transaction_data_by_minute_binary = transaction_data_by_minute.applymap(lambda x: 1 if x > 0 else 0)

transaction_data_by_minute_binary

分类名称,水生根茎类,花叶类,花菜类,茄类,辣椒类,食用菌
完整时间戳,,,,,,
2020-07-01 09:15:00,0,0,0,0,1,0
2020-07-01 09:17:00,0,1,0,0,1,0
2020-07-01 09:19:00,0,1,0,0,0,0
2020-07-01 09:20:00,0,1,0,0,0,0
2020-07-01 09:21:00,0,1,0,0,1,0
...,...,...,...,...,...,...
2023-06-30 21:26:00,0,0,0,0,0,1
2023-06-30 21:32:00,0,0,0,0,0,1
2023-06-30 21:34:00,0,0,1,0,0,0


In [6]:
from mlxtend.frequent_patterns import apriori
transaction_data_by_minute_binary = transaction_data_by_minute_binary.astype(bool)
frequent_itemsets = apriori(transaction_data_by_minute_binary, min_support=0.01, use_colnames=True)


In [7]:
frequent_itemsets

,support,itemsets
0,0.134360,(水生根茎类)
1,0.574949,(花叶类)
2,0.199415,(花菜类)
3,0.110064,(茄类)
4,0.408108,(辣椒类)
5,0.304047,(食用菌)
6,0.069858,"(花叶类, 水生根茎类)"
7,0.028273,"(花菜类, 水生根茎类)"
8,0.013292,"(茄类, 水生根茎类)"
9,0.050560,"(辣椒类, 水生根茎类)"


In [8]:
from mlxtend.frequent_patterns import association_rules

rules = association_rules(frequent_itemsets, metric='lift', min_threshold=1)

In [9]:
rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
0,(花菜类),(水生根茎类),0.199415,0.134360,0.028273,0.141777,1.055200,0.001479,1.008642,0.065343
1,(水生根茎类),(花菜类),0.134360,0.199415,0.028273,0.210423,1.055200,0.001479,1.013941,0.060432
2,(食用菌),(水生根茎类),0.304047,0.134360,0.042264,0.139005,1.034572,0.001412,1.005395,0.048016
3,(水生根茎类),(食用菌),0.134360,0.304047,0.042264,0.314559,1.034572,0.001412,1.015336,0.038604
4,(茄类),(辣椒类),0.110064,0.408108,0.047555,0.432063,1.058699,0.002637,1.042180,0.062302
...,...,...,...,...,...,...,...,...,...,...
133,"(辣椒类, 食用菌)","(茄类, 花叶类)",0.111673,0.057500,0.011012,0.098610,1.714954,0.004591,1.045607,0.469302
134,(茄类),"(花叶类, 食用菌, 辣椒类)",0.110064,0.067188,0.011012,0.100052,1.489132,0.003617,1.036517,0.369091
135,(花叶类),"(茄类, 辣椒类, 食用菌)",0.574949,0.016179,0.011012,0.019153,1.183857,0.001710,1.003033,0.365376
136,(食用菌),"(茄类, 花叶类, 辣椒类)",0.304047,0.028588,0.011012,0.036218,1.266929,0.002320,1.007918,0.302736


In [14]:
# 筛选提升度大于1的规则
rules_high_lift = rules[rules['lift'] > 1].sort_values(by='lift', ascending=False)
rules_high_lift

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
132,"(花叶类, 辣椒类)","(茄类, 食用菌)",0.199932,0.029925,0.011012,0.055079,1.840576,0.005029,1.026620,0.570816
129,"(茄类, 食用菌)","(花叶类, 辣椒类)",0.029925,0.199932,0.011012,0.367990,1.840576,0.005029,1.265910,0.470780
100,"(花叶类, 食用菌)","(辣椒类, 水生根茎类)",0.152049,0.050560,0.013749,0.090424,1.788455,0.006061,1.043827,0.519910
105,"(辣椒类, 水生根茎类)","(花叶类, 食用菌)",0.050560,0.152049,0.013749,0.271933,1.788455,0.006061,1.164661,0.464335
102,"(花叶类, 水生根茎类)","(辣椒类, 食用菌)",0.069858,0.111673,0.013749,0.196814,1.762421,0.005948,1.106005,0.465089
...,...,...,...,...,...,...,...,...,...,...
75,(花叶类),"(辣椒类, 食用菌)",0.574949,0.111673,0.067188,0.116859,1.046442,0.002982,1.005873,0.104412
61,"(茄类, 辣椒类)",(花叶类),0.047555,0.574949,0.028588,0.601151,1.045572,0.001246,1.065693,0.045762
64,(花叶类),"(茄类, 辣椒类)",0.574949,0.047555,0.028588,0.049722,1.045572,0.001246,1.002281,0.102543
3,(水生根茎类),(食用菌),0.134360,0.304047,0.042264,0.314559,1.034572,0.001412,1.015336,0.038604
